In [ ]:
from langchain_classic.agents import create_openai_tools_agent
!pip install arxiv


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [4]:

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
wiki.name


'wikipedia'

In [6]:
import os
import requests
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. SETUP
DB_PATH = "faiss_index_web"

# 2. LOGIC: Initialize Models (Using llama3.2 is safer for CUDA errors)
# If llama3 continues to crash, change "llama3" to "llama3.2" below.
embeddings = OllamaEmbeddings(model="nomic-embed-text")
llm = ChatOllama(model="llama3.2", temperature=0)

# 3. LOGIC: Load Database
if os.path.exists(DB_PATH):
    vectordb = FAISS.load_local(DB_PATH, embeddings, allow_dangerous_deserialization=True)
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})
    print("✅ Database loaded successfully.")
else:
    print("❌ Error: Run your scraping code first to create the index!")

# 4. LOGIC: Build the Chain
template = """Answer the question based ONLY on the following context:
{context}

Question: {input}
Answer:"""

prompt = ChatPromptTemplate.from_template(template)
format_docs = lambda docs: "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 5. EXECUTION with Error Handling
print("\n--- SYSTEM READY ---")
try:
    print("AI EXPLANATION:")
    print(rag_chain.invoke("What is LangSmith?"))
except Exception as e:
    print(f"⚠️ GPU Error detected. Try running 'ollama pull llama3.2' and changing the model name in the code.")
    print(f"Detailed Error: {e}")

✅ Database loaded successfully.

--- SYSTEM READY ---
AI EXPLANATION:
LangSmith is a tool for developing, debugging, and deploying Large Language Model (LLM) applications. It provides a framework-agnostic platform that allows users to prototype locally, deploy to production with integrated monitoring and evaluation, and build more reliable AI systems.


In [7]:
# Logic: Import from 'langchain_core' because it's already in your memory
from langchain_core.tools.retriever import create_retriever_tool

# 2. Logic: Create the Tool
# This connects your PDF 'retriever' to a tool an Agent can use
retriever_tool = create_retriever_tool(
    retriever,
    'langsmith_search',
    "Use this tool to search for technical details in the 'Attention Is All You Need' PDF."
)

print("✅ Success: Tool is created and ready!")

✅ Success: Tool is created and ready!


In [8]:
retriever_tool.name

'langsmith_search'

In [9]:
import arxiv
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

# Create the specific tool
arxiv_tool = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=1))

# ✅ Logic: Use the variable YOU created, not the library name
print(arxiv_tool.name)

arxiv


In [10]:
tools=[wiki,arxiv,retriever_tool]


In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'D:\\Langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 <module 'arxiv' from 'D:\\Langchain\\venv\\Lib\\site-packages\\arxiv\\__init__.py'>,
 StructuredTool(name='langsmith_search', description="Use this tool to search for technical details in the 'Attention Is All You Need' PDF.", args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x000002BCC2C1D940>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x000002BCC2C1DBC0>)]

In [12]:
# 1. Logic: Use the dedicated Ollama package (Standard for 2026)
from langchain_ollama import ChatOllama

# 2. Logic: Future-Proofing
# No API Keys needed! Safe from leaks and charges.
llm = ChatOllama(
    model="llama3",
    temperature=0,
    # This ensures the model stays consistent
)

print("✅ Success: Local LLM initialized. No OpenAI Key required.")

✅ Success: Local LLM initialized. No OpenAI Key required.


In [1]:
import os
import langchainhub as hub
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.tools import Tool

# ✅ Logic: These are the correct paths for LangChain 1.x
from langchain.agents import AgentExecutor
from langchain.agents.react.agent import create_react_agent

# 1. Initialize Models
llm = ChatOllama(model="llama3", temperature=0)
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# 2. Load Memory
if os.path.exists("faiss_index"):
    db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    retriever = db.as_retriever()
else:
    print("❌ ERROR: Run your DB saving code first!")

# 3. Define the Tool
def search_pdf(query: str):
    docs = retriever.invoke(query)
    return "\n\n".join([d.page_content for d in docs])

retriever_tool = Tool(
    name="search_pdf_tool",
    func=search_pdf,
    description="Use this for technical questions about the Attention paper."
)
tools = [retriever_tool]

# 4. Pull Prompt & Create Agent
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)

# 5. Build the Executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 6. Run it
print("\n--- AGENT STARTING ---")
query = "What is the formula for Scaled Dot-Product Attention?"
# Logic: Always use a dictionary for Agents
response = agent_executor.invoke({"input": query})

print("\n--- FINAL ANSWER ---")
print(response["output"])

ImportError: cannot import name 'AgentExecutor' from 'langchain.agents' (D:\Langchain\venv\Lib\site-packages\langchain\agents\__init__.py)